In [18]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from resnet_rn06_eembc import * 
import qkeras
!pwd
model = qkeras.utils.load_qmodel('old_training/trained_model/deploy/resnet_v1_eembc_RN06_bilinear/small_model_best.h5')
#model = qkeras.utils.load_qmodel('resnet_v1_eembc_RN06_PokeV2/small_model_best.h5')
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score, accuracy_score, top_k_accuracy_score
batch_size=32

/home/bhawks/RN0X_Pokemon


In [2]:
datagen = ImageDataGenerator(
    rotation_range=60,
    width_shift_range=0.15,
    height_shift_range=0.15,
    brightness_range=(0.8,1.5),
    zoom_range = [1.0, 1.3], 
    horizontal_flip=True,
    validation_split=0.25,
    rescale=1./255 # normalize values to between 0-1
    # preprocessing_function=random_crop,
    #brightness_range=(0.9, 1.2),
    #contrast_range=(0.9, 1.2)
)
tdatagen = ImageDataGenerator(rescale=1./255) # No augmentation
tdatagen2 = ImageDataGenerator( # all augments, but minimum zoon to reduce interference of top bar?
    rotation_range=60,
    width_shift_range=0.15,
    height_shift_range=0.15,
    brightness_range=(0.8,1.5),
    zoom_range = [1.10, 1.3],
    horizontal_flip=True,
    validation_split=0.25,
    rescale=1./255
    # normalize values to between 0-1
    # preprocessing_function=random_crop,
    #brightness_range=(0.9, 1.2),
    #contrast_range=(0.9, 1.2)
)
tdatagen3 = ImageDataGenerator( # no augments, but minimum zoon to reduce interference of top bar?
    #zoom_range = [1.10, 1.3],
    rescale=1./255 # normalize values to between 0-1,
    # preprocessing_function=random_crop,
    #brightness_range=(0.9, 1.2),
    #contrast_range=(0.9, 1.2)
)
train_generator = datagen.flow_from_directory('./data/SmallPokemonData/',
                            target_size=(32,32),
                            batch_size=batch_size,
                            color_mode='rgb',
                            class_mode='categorical',
                            interpolation='bilinear',
                            subset='training',
                            keep_aspect_ratio=True,
                            save_to_dir="/home/bhawks/RN0X_Pokemon/data/debug/datagen_Train/"                                             
                            )

validation_generator = datagen.flow_from_directory('./data/SmallPokemonData/',
                            target_size=(32,32),
                            batch_size=batch_size,
                            color_mode='rgb',
                            class_mode='categorical',
                            interpolation='bilinear',
                            subset='validation',
                            keep_aspect_ratio=True,
                            shuffle=False,
                            save_to_dir="/home/bhawks/RN0X_Pokemon/data/debug/datagen_Val/"
                            )    
test_generator = tdatagen.flow_from_directory('./data/PokeCard/',
                        target_size=(32,32),
                        batch_size=batch_size,
                        color_mode='rgb',
                        class_mode='categorical',
                        interpolation='bilinear',
                        shuffle=False,
                        save_to_dir="/home/bhawks/RN0X_Pokemon/data/debug/datagen_test_NoAug/",
                        keep_aspect_ratio=True,)#,
                        #keep_aspect_ratio=True)

test2_generator = tdatagen2.flow_from_directory('./data/PokeCard/',
                        target_size=(32,32),
                        batch_size=batch_size,
                        color_mode='rgb',
                        class_mode='categorical',
                        interpolation='bilinear',
                        shuffle=False,
                        save_to_dir="/home/bhawks/RN0X_Pokemon/data/debug/datagen_test_fullAug/",
                            keep_aspect_ratio=True,)#,
                        #keep_aspect_ratio=True)

test3_generator = tdatagen3.flow_from_directory('./data/PokeCard/',
                        target_size=(32,32),
                        batch_size=batch_size,
                        color_mode='rgb',
                        class_mode='categorical',
                        interpolation='bilinear',
                        shuffle=False,
                        save_to_dir="/home/bhawks/RN0X_Pokemon/data/debug/datagen_test_ZoomOnly/",
                            keep_aspect_ratio=True,)#,
                        #keep_aspect_ratio=True)

Found 1147 images belonging to 10 classes.
Found 377 images belonging to 10 classes.
Found 574 images belonging to 10 classes.
Found 574 images belonging to 10 classes.
Found 574 images belonging to 10 classes.


In [3]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 q_conv2d_batchnorm (QConv2  (None, 8, 8, 32)          1025      
 DBatchnorm)                                                     
                                                                 
 q_activation (QActivation)  (None, 8, 8, 32)          0         
                                                                 
 q_conv2d_batchnorm_1 (QCon  (None, 8, 8, 32)          9377      
 v2DBatchnorm)                                                   
                                                                 
 q_activation_1 (QActivatio  (None, 8, 8, 32)          0         
 n)                                                              
                                                             

In [4]:
data_list = []
batch_index = 0
while batch_index <= test_generator.batch_index:
    data = test_generator.next()
    data_list.append(data[0])
    batch_index = batch_index + 1
print(len(data_list))

18


In [5]:
score = model.evaluate(test_generator)
print(score)

18/18 [==============================] - 5s 242ms/step - loss: 1.4131 - accuracy: 0.6533
[1.4130908250808716, 0.6533101201057434]


In [6]:
y_pred = model.predict(test_generator)
# evaluate with test dataset and share same prediction results
evaluation = model.evaluate(test_generator)
print(evaluation) 
y_test = test_generator.classes
print(y_pred[0])
print(y_test)
score = top_k_accuracy_score(y_test, y_pred, k=2)
print(score)

18/18 [==============================] - 5s 250ms/step - loss: 1.4131 - accuracy: 0.6533
[1.4130908250808716, 0.6533101201057434]
[9.99998808e-01 1.06155070e-12 1.03272134e-10 6.12947071e-09
 3.39609812e-10 1.80155162e-08 1.44215903e-07 5.73674476e-11
 1.29318130e-08 1.08398717e-06]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 

In [7]:
from sklearn.metrics import classification_report, confusion_matrix
print(len(y_pred))
print(len(y_test))
print('Confusion Matrix - Test Generator')
print(confusion_matrix(y_test, np.argmax(y_pred, axis=1)))
print('Classification Report - Test Generator')
target_names = test_generator.class_indices.keys() #['bulbasaur', 'charmander', 'eevee', 'gengar', 'jigglypuff', 'mewtwo', 'onix', 'pikachu', 'snorlax', 'squirtle']

print(classification_report(y_test, np.argmax(y_pred, axis=1), target_names=target_names))

574
574
Confusion Matrix - Test Generator
[[ 18   0   0   0   0   0   3   1   0   3]
 [  0  34   1   0   0   0   1   5   1   0]
 [  0  17  45   0   0   1   8   1   4   0]
 [  0   0   1  35   0   4   2   0   1   0]
 [  0   1   0   0  30   0   2   0   0   1]
 [  1   5   3  10   5  33  17   0   2   3]
 [  0   1   0   1   1   1  18   2   3   2]
 [  0  15  18   1   2   0  10 110  10   4]
 [  0   1   1   2   1   0   5   1  36   2]
 [  1   2   2   0   0   0   0   2   4  16]]
Classification Report - Test Generator
              precision    recall  f1-score   support

   Bulbasaur       0.90      0.72      0.80        25
  Charmander       0.45      0.81      0.58        42
       Eevee       0.63      0.59      0.61        76
      Gengar       0.71      0.81      0.76        43
  Jigglypuff       0.77      0.88      0.82        34
      Mewtwo       0.85      0.42      0.56        79
        Onix       0.27      0.62      0.38        29
     Pikachu       0.90      0.65      0.75       170
 

In [19]:
score2 = model.evaluate(test2_generator)
print(score2)

18/18 [==============================] - 6s 291ms/step - loss: 2.2142 - accuracy: 0.6150
[2.2141733169555664, 0.6149826049804688]


In [20]:
    y_pred = model.predict(test2_generator)
    # evaluate with test dataset and share same prediction results
    evaluation = model.evaluate(test2_generator)
    print(evaluation) 
    y_test = test2_generator.classes
    print(y_pred[0])
    print(y_test)
    score = top_k_accuracy_score(y_test, y_pred, k=2)
    print(score)

18/18 [==============================] - 5s 288ms/step - loss: 1.9488 - accuracy: 0.6272
[1.9487733840942383, 0.6271777153015137]
[9.9892861e-01 2.2417657e-05 4.1883386e-08 2.6883960e-08 3.1207095e-10
 5.2935520e-06 1.3366597e-05 4.2807464e-09 1.9790218e-06 1.0281664e-03]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7

In [21]:
from sklearn.metrics import classification_report, confusion_matrix
print(len(y_pred))
print(len(y_test))
print('Confusion Matrix - Test2 Generator')
print(confusion_matrix(y_test, np.argmax(y_pred, axis=1)))
print('Classification Report - Test2 Generator')
target_names = test2_generator.class_indices.keys() #['bulbasaur', 'charmander', 'eevee', 'gengar', 'jigglypuff', 'mewtwo', 'onix', 'pikachu', 'snorlax', 'squirtle']

print(classification_report(y_test, np.argmax(y_pred, axis=1), target_names=target_names))

574
574
Confusion Matrix - Test2 Generator
[[ 16   0   0   0   0   0   1   3   0   5]
 [  0  30   2   0   0   0   1   8   0   1]
 [  2  15  39   0   1   2   2  12   2   1]
 [  0   2   0  22   1  15   1   0   0   2]
 [  0   1   1   1  17   5   1   0   4   4]
 [  0   6   1   1   2  54   5   3   0   7]
 [  1   0   0   1   1   3  16   5   1   1]
 [  2  10   2   0   0   1   4 137   2  12]
 [  2   0   1   0   0   2   0   2  28  14]
 [  1   2   0   1   0   0   0   5   0  18]]
Classification Report - Test2 Generator
              precision    recall  f1-score   support

   Bulbasaur       0.67      0.64      0.65        25
  Charmander       0.45      0.71      0.56        42
       Eevee       0.85      0.51      0.64        76
      Gengar       0.85      0.51      0.64        43
  Jigglypuff       0.77      0.50      0.61        34
      Mewtwo       0.66      0.68      0.67        79
        Onix       0.52      0.55      0.53        29
     Pikachu       0.78      0.81      0.79       170

In [11]:
y_pred = model.predict(test3_generator)
# evaluate with test dataset and share same prediction results
evaluation = model.evaluate(test3_generator)
print(evaluation) 
y_test = test3_generator.classes
print(y_pred[0])
print(y_test)
score = top_k_accuracy_score(y_test, y_pred, k=2)
print(score)

18/18 [==============================] - 4s 236ms/step - loss: 1.4131 - accuracy: 0.6533
[1.4130908250808716, 0.6533101201057434]
[9.99998808e-01 1.06155070e-12 1.03272134e-10 6.12947071e-09
 3.39609812e-10 1.80155162e-08 1.44215903e-07 5.73674476e-11
 1.29318130e-08 1.08398717e-06]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 

In [12]:
from sklearn.metrics import classification_report, confusion_matrix
print(len(y_pred))
print(len(y_test))
print('Confusion Matrix - Test3 Generator')
print(confusion_matrix(y_test, np.argmax(y_pred, axis=1)))
print('Classification Report- Test3 Generator')
target_names = test3_generator.class_indices.keys() #['bulbasaur', 'charmander', 'eevee', 'gengar', 'jigglypuff', 'mewtwo', 'onix', 'pikachu', 'snorlax', 'squirtle']

print(classification_report(y_test, np.argmax(y_pred, axis=1), target_names=target_names))

574
574
Confusion Matrix - Test3 Generator
[[ 18   0   0   0   0   0   3   1   0   3]
 [  0  34   1   0   0   0   1   5   1   0]
 [  0  17  45   0   0   1   8   1   4   0]
 [  0   0   1  35   0   4   2   0   1   0]
 [  0   1   0   0  30   0   2   0   0   1]
 [  1   5   3  10   5  33  17   0   2   3]
 [  0   1   0   1   1   1  18   2   3   2]
 [  0  15  18   1   2   0  10 110  10   4]
 [  0   1   1   2   1   0   5   1  36   2]
 [  1   2   2   0   0   0   0   2   4  16]]
Classification Report- Test3 Generator
              precision    recall  f1-score   support

   Bulbasaur       0.90      0.72      0.80        25
  Charmander       0.45      0.81      0.58        42
       Eevee       0.63      0.59      0.61        76
      Gengar       0.71      0.81      0.76        43
  Jigglypuff       0.77      0.88      0.82        34
      Mewtwo       0.85      0.42      0.56        79
        Onix       0.27      0.62      0.38        29
     Pikachu       0.90      0.65      0.75       170


In [13]:
y_pred = model.predict(validation_generator)
# evaluate with test dataset and share same prediction results
evaluation = model.evaluate(validation_generator)
print(evaluation) 
y_test = validation_generator.classes
print(y_pred[0])
print(y_test)
score = top_k_accuracy_score(y_test, y_pred, k=2)
print(score)

 2/12 [====>.........................] - ETA: 9s 

/home/bhawks/.conda/envs/poke/lib/python3.8/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


12/12 [==============================] - 6s 421ms/step - loss: 0.4492 - accuracy: 0.8727
[0.4491543173789978, 0.8726790547370911]
[9.99999642e-01 3.71272367e-18 1.01263738e-14 1.03969305e-14
 4.64664277e-13 8.16828549e-12 2.38120592e-07 2.20599125e-12
 4.29453001e-11 8.88055567e-08]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 

In [14]:
print(len(y_pred))
print(len(y_test))
print( np.argmax(y_pred, axis=1))
print( y_test)
print('Confusion Matrix - Validation Data')
print(confusion_matrix(y_test, np.argmax(y_pred, axis=1)))
print('Classification Report - Validation Data')
target_names = validation_generator.class_indices.keys() #[str(i) for i in range(10)] # #['bulbasaur', 'charmander', 'eevee', 'gengar', 'jigglypuff', 'mewtwo', 'onix', 'pikachu', 'snorlax', 'squirtle']

print(classification_report(y_test, np.argmax(y_pred, axis=1), target_names=target_names))

377
377
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 2 1 1 4 1 1 2 2 1 1 8 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 5 4 7 1 2
 2 1 2 2 2 1 7 2 2 7 2 2 2 1 2 2 8 2 2 2 7 2 7 2 6 2 2 2 2 2 2 1 2 3 9 6 3
 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 6 6 5 5 6 6 5 3 5 6 6 5 5 3 5 6 5 5 6 3 6 5 5 5 5 5 5 5 5 5 5 5 6
 5 5 5 5 5 5 5 4 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 5 3 6 6 4 6 6 6 6 6 6 6 7
 7 7 7 2 7 7 1 7 7 7 7 7 7 7 7 7 7 7 7 7 7 2 7 7 7 7 2 7 7 7 7 7 2 7 7 7 7
 7 7 7 7 7 7 2 7 7 7 7 7 4 8 8 8 8 0 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
 8 2 8 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 8 9
 9 9 9 9 9 9 9]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3
 